# Preprocess Spain and Basque Country buildings

In [2]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import os

## Spain

In [3]:
# Folder containing the GML files
folder = Path("/data/uscuni-ulce/extension/spain/")

# Find all .gml files
gml_files = list(folder.glob("*.gml"))

# Read and concatenate into one GeoDataFrame
gdfs = [gpd.read_file(f,columns = ["geometry","beginning", "end","value","currentUse"],use_arrow=True).to_crs("EPSG:3035") for f in gml_files]

In [10]:
spain_raw = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True)).explode()
spain_raw

beginning                  end          currentUse  value  \
0         1944-01-01T00:00:00  1944-01-01T00:00:00  4_3_publicServices     55   
1         1944-01-01T00:00:00  1944-01-01T00:00:00        3_industrial     76   
2         1900-01-01T00:00:00  2000-01-01T00:00:00  4_3_publicServices    119   
3         2000-01-01T00:00:00  2000-01-01T00:00:00       2_agriculture   1047   
3         2000-01-01T00:00:00  2000-01-01T00:00:00       2_agriculture   1047   
...                       ...                  ...                 ...    ...   
12474033  1943-01-01T00:00:00  2000-01-01T00:00:00       1_residential    588   
12474034  1943-01-01T00:00:00  1943-01-01T00:00:00        3_industrial    612   
12474035  1943-01-01T00:00:00  1943-01-01T00:00:00       1_residential    198   
12474036  1943-01-01T00:00:00  2003-01-01T00:00:00       1_residential    414   
12474036  1943-01-01T00:00:00  2003-01-01T00:00:00       1_residential    414   

                                                   geometry  
0         POLYGON ((3328786.428 2102374.993, 3328777.707...  
1         POLYGON ((3328723.849 2101563.942, 3328724.862...  
2         POLYGON ((3324585.802 2104952.066, 3324588.398...  
3         POLYGON ((3325928.207 2102926.23, 3325920.92 2...  
3         POLYGON ((3326027.383 2102950.242, 3326036.09 ...  
...                                                     ...  
12474033  POLYGON ((3325293.988 2132331.395, 3325291.564...  
12474034  POLYGON ((3325301.591 2132309.053, 3325303.907...  
12474035  POLYGON ((3325315.231 2132297.456, 3325321.111...  
12474036  POLYGON ((3325329.104 2132292.959, 3325328.34 ...  
12474036  POLYGON ((3325346.548 2132296.63, 3325351.578 ...  

[16582745 rows x 5 columns]

In [24]:
# Compute area in meters (ensure projected CRS)
area_mask = spain_raw.area > 50000

# CurrentUse is public services or NaN
use_mask = (spain_raw["currentUse"] == "4_3_publicServices") | (spain_raw["currentUse"].isna())

# Drop rows matching both conditions
selected = spain_raw.drop(spain_raw[area_mask & use_mask].index)

In [29]:
selected=selected.rename(columns = {"value":"floor_area"})

In [30]:
selected

beginning                  end          currentUse  \
0         1944-01-01T00:00:00  1944-01-01T00:00:00  4_3_publicServices   
1         1944-01-01T00:00:00  1944-01-01T00:00:00        3_industrial   
2         1900-01-01T00:00:00  2000-01-01T00:00:00  4_3_publicServices   
3         2000-01-01T00:00:00  2000-01-01T00:00:00       2_agriculture   
3         2000-01-01T00:00:00  2000-01-01T00:00:00       2_agriculture   
...                       ...                  ...                 ...   
12474033  1943-01-01T00:00:00  2000-01-01T00:00:00       1_residential   
12474034  1943-01-01T00:00:00  1943-01-01T00:00:00        3_industrial   
12474035  1943-01-01T00:00:00  1943-01-01T00:00:00       1_residential   
12474036  1943-01-01T00:00:00  2003-01-01T00:00:00       1_residential   
12474036  1943-01-01T00:00:00  2003-01-01T00:00:00       1_residential   

          floor_area                                           geometry  
0                 55  POLYGON ((3328786.428 2102374.993, 3328777.707...  
1                 76  POLYGON ((3328723.849 2101563.942, 3328724.862...  
2                119  POLYGON ((3324585.802 2104952.066, 3324588.398...  
3               1047  POLYGON ((3325928.207 2102926.23, 3325920.92 2...  
3               1047  POLYGON ((3326027.383 2102950.242, 3326036.09 ...  
...              ...                                                ...  
12474033         588  POLYGON ((3325293.988 2132331.395, 3325291.564...  
12474034         612  POLYGON ((3325301.591 2132309.053, 3325303.907...  
12474035         198  POLYGON ((3325315.231 2132297.456, 3325321.111...  
12474036         414  POLYGON ((3325329.104 2132292.959, 3325328.34 ...  
12474036         414  POLYGON ((3325346.548 2132296.63, 3325351.578 ...  

[16582013 rows x 5 columns]

In [31]:
selected.to_parquet("/data/uscuni-ulce/extension/spain/clean_3035.parquet")

## Araba/Alava

In [3]:
!ls "/data/uscuni-ulce/extension/basque/araba_alava"

ES.AFA.BU.0101_3042.gml  ES.AFA.BU.0123_3042.gml  ES.AFA.BU.0147_3042.gml
ES.AFA.BU.0102_3042.gml  ES.AFA.BU.0126_3042.gml  ES.AFA.BU.0149_3042.gml
ES.AFA.BU.0103_3042.gml  ES.AFA.BU.0127_3042.gml  ES.AFA.BU.0151_3042.gml
ES.AFA.BU.0104_3042.gml  ES.AFA.BU.0128_3042.gml  ES.AFA.BU.0152_3042.gml
ES.AFA.BU.0106_3042.gml  ES.AFA.BU.0130_3042.gml  ES.AFA.BU.0153_3042.gml
ES.AFA.BU.0108_3042.gml  ES.AFA.BU.0131_3042.gml  ES.AFA.BU.0154_3042.gml
ES.AFA.BU.0109_3042.gml  ES.AFA.BU.0132_3042.gml  ES.AFA.BU.0155_3042.gml
ES.AFA.BU.0110_3042.gml  ES.AFA.BU.0133_3042.gml  ES.AFA.BU.0156_3042.gml
ES.AFA.BU.0111_3042.gml  ES.AFA.BU.0134_3042.gml  ES.AFA.BU.0157_3042.gml
ES.AFA.BU.0113_3042.gml  ES.AFA.BU.0135_3042.gml  ES.AFA.BU.0158_3042.gml
ES.AFA.BU.0114_3042.gml  ES.AFA.BU.0136_3042.gml  ES.AFA.BU.0159_3042.gml
ES.AFA.BU.0116_3042.gml  ES.AFA.BU.0137_3042.gml  ES.AFA.BU.0160_3042.gml
ES.AFA.BU.0117_3042.gml  ES.AFA.BU.0139_3042.gml  ES.AFA.BU.0161_3042.gml
ES.AFA.BU.0118_3042.gml  ES.AFA.BU.014

In [11]:
# Folder containing the GML files
folder = Path("/data/uscuni-ulce/extension/basque/araba_alava")

# Find all .gml files
gml_files = list(folder.glob("*.gml"))

# Read and concatenate into one GeoDataFrame
gdfs = [gpd.read_file(f,columns = ["geometry","anyPoint", "value"],use_arrow=True).to_crs("EPSG:3035") for f in gml_files]

In [12]:
araba_alava = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
araba_alava

anyPoint  value  \
0      1997-12-31T00:00:00      3   
1      2009-12-31T00:00:00      0   
2      1970-12-31T00:00:00      3   
3      1997-12-31T00:00:00      6   
4      2001-12-31T00:00:00      3   
...                    ...    ...   
81257  2004-12-31T00:00:00      3   
81258  2008-12-31T00:00:00      6   
81259  1800-12-31T00:00:00      6   
81260  1800-12-31T00:00:00      6   
81261  1960-12-31T00:00:00      3   

                                                geometry  
0      POLYGON ((3311074.159 2265588.406, 3311069.784...  
1      POLYGON ((3310798.778 2265145.178, 3310798.903...  
2      POLYGON ((3311051.897 2265045.697, 3311071.174...  
3      POLYGON ((3311071.662 2265047.478, 3311072.765...  
4      POLYGON ((3311198.001 2265026.355, 3311216.489...  
...                                                  ...  
81257  POLYGON ((3301872.91 2257088.536, 3301870.612 ...  
81258  POLYGON ((3302975.665 2258112.935, 3302974.023...  
81259  POLYGON ((3300315.89 2254941.917, 3300316.288 ...  
81260  POLYGON ((3300260.191 2254767.308, 3300261.799...  
81261  POLYGON ((3300249.99 2254782.883, 3300249.846 ...  

[81262 rows x 3 columns]

In [15]:
araba_alava = araba_alava.rename(columns={"value":"height","anyPoint":"beginning"})
araba_alava.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

In [16]:
araba_alava.to_parquet("/data/uscuni-ulce/extension/basque/araba_alava/clean_3035.parquet")

## Bizkaia

In [8]:
sample = gpd.read_file("/data/uscuni-ulce/extension/basque/bizkaia/ES.BFA.BU.001.gml")
sample

gml_id                                         identifier  \
0          BuildingS.1  https://geo.bizkaia.eus/arcgisserverinspire/re...   
1          BuildingS.2  https://geo.bizkaia.eus/arcgisserverinspire/re...   
2          BuildingS.3  https://geo.bizkaia.eus/arcgisserverinspire/re...   
3          BuildingS.4  https://geo.bizkaia.eus/arcgisserverinspire/re...   
4          BuildingS.6  https://geo.bizkaia.eus/arcgisserverinspire/re...   
...                ...                                                ...   
1351  BuildingS.682840  https://geo.bizkaia.eus/arcgisserverinspire/re...   
1352  BuildingS.682971  https://geo.bizkaia.eus/arcgisserverinspire/re...   
1353  BuildingS.683219  https://geo.bizkaia.eus/arcgisserverinspire/re...   
1354  BuildingS.683522  https://geo.bizkaia.eus/arcgisserverinspire/re...   
1355  BuildingS.683668  https://geo.bizkaia.eus/arcgisserverinspire/re...   

     beginLifespanVersion anyPoint beginning                  end  end_  \
0     2011-07-14T15:28:37     None      None  1986-12-31T00:00:00  None   
1     2013-12-19T08:45:15     None      None  1963-12-31T00:00:00  None   
2     2013-12-19T08:45:15     None      None  1960-12-31T00:00:00  None   
3     2013-12-19T08:45:15     None      None  1963-12-31T00:00:00  None   
4     2017-05-30T12:02:58     None      None  1970-12-31T00:00:00  None   
...                   ...      ...       ...                  ...   ...   
1351  2025-05-28T08:02:03     None      None  1975-12-31T00:00:00  None   
1352  2025-06-06T10:47:52     None      None  1920-12-31T00:00:00  None   
1353  2025-06-26T10:41:00     None      None  2016-12-31T00:00:00  None   
1354  2025-07-11T13:11:16     None      None  1800-12-31T00:00:00  None   
1355  2025-07-29T12:27:11     None      None  1995-12-31T00:00:00  None   

     elevation endLifespanVersion  \
0         None               None   
1         None               None   
2         None               None   
3         None               None   
4         None               None   
...        ...                ...   
1351      None               None   
1352      None               None   
1353      None               None   
1354      None               None   
1355      None               None   

                                      informationSystem  ... percentage  \
0     https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
1     https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
2     https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
3     https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
4     https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
...                                                 ...  ...        ...   
1351  https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
1352  https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
1353  https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
1354  https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   
1355  https://appsec.ebizkaia.eus/O4GC000C/vistas/vi...  ...       None   

     numberOfDwellings numberOfBuildingUnits numberOfFloorsAboveGround parts  \
0                  NaN                    16                       3.0  None   
1                  8.0                    11                       6.0  None   
2                 18.0                    23                       7.0  None   
3                  8.0                    13                       6.0  None   
4                 12.0                    15                       5.0  None   
...                ...                   ...                       ...   ...   
1351               1.0                     1                       3.0  None   
1352               3.0                     3                       4.0  None   
1353               1.0                     1                       3.0  None   
1354               1.0                     1